In [1]:
import os
import random
from PIL import Image, ImageFilter
import numpy as np
import cv2
from tqdm import tqdm

In [2]:
folder_path = "../8classes_4468/"

# 列出資料夾中的所有項目
items = os.listdir(folder_path)

# 過濾出資料夾
subfolders = [item for item in items if os.path.isdir(os.path.join(folder_path, item))]

# 將資料夾名稱放入列表
folder_names = subfolders

print("資料夾名稱列表：", folder_names)

資料夾名稱列表： ['CARRIER1_Door Damage', 'CARRIER2_Panel dents or scratches', 'CARRIER3_Panel aging, rust or breach', 'CARRIER4_Placards removal', 'SHIPPER1_Floor stains, contamination or cracks', 'SHIPPER2_Panel or Sidewall protrusion', 'SHIPPER3_Cross member or forklift pocket overweight-deformation', 'SHIPPER4_Panel or Sidewalls dirts']


In [3]:
# 定義資料夾名稱列表和模糊類型列表
folder_names = ['CARRIER1_Door Damage', 'CARRIER2_Panel dents or scratches', 'CARRIER3_Panel aging, rust or breach', 
                'CARRIER4_Placards removal', 'SHIPPER1_Floor stains, contamination or cracks', 
                'SHIPPER2_Panel or Sidewall protrusion', 'SHIPPER3_Cross member or forklift pocket overweight-deformation',
                'SHIPPER4_Panel or Sidewalls dirts']
blur_types = ["Gaussian", "Median", "Mean", "Bilateral", "Custom"]

# 定義自定義模糊核
custom_kernel = np.array([
    [0, 0, 0.1, 0, 0],
    [0, 0.1, 0.5, 0.1, 0],
    [0.1, 0.5, 1, 0.5, 0.1],
    [0, 0.1, 0.5, 0.1, 0],
    [0, 0, 0.1, 0, 0]
])

# 正規化模糊核，以確保模糊後的圖像不會變亮或變暗
custom_kernel /= np.sum(custom_kernel)

# 迴圈遍歷資料夾名稱和模糊類型
for folder_name in folder_names:
    for blur_type in blur_types:
        # 設定資料夾和目標資料夾路徑
        image_folder = f"./{folder_name}/"
        target_folder = f"./blurred_image2/{folder_name}_{blur_type}/"
        os.makedirs(target_folder, exist_ok=True)
        
        # 讀取資料夾中的所有圖片
        image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
        # 從中隨機挑選100張圖片，如果圖片數量少於100，則隨機選取，並補足到100張
        if len(image_files) >= 100:
            selected_images = random.sample(image_files, 100)
        else:
            # 如果圖片數量少於100，先將所有圖片加入選取列表
            selected_images = image_files.copy()
            # 再從剩餘的圖片中隨機選取，直到補足100張
            while len(selected_images) < 100:
                additional_images = random.sample(image_files, min(100 - len(selected_images), len(image_files)))
                selected_images.extend(additional_images)
        
        # 使用 tqdm 來追蹤生成進度
        with tqdm(total=len(selected_images), desc=f"Processing {folder_name} - {blur_type}") as pbar:
            # 對每張圖片進行模糊處理並保存到目標資料夾中
            for image_file in selected_images:
                # 讀取圖片
                image_path = os.path.join(image_folder, image_file)
                image = Image.open(image_path)
                
                # 根據模糊類型進行處理
                if blur_type == "Gaussian":
                    # 高斯模糊處理
                    blurred_image = image.filter(ImageFilter.GaussianBlur(radius=random.randint(2, 10)))
                elif blur_type == "Median":
                    filter_size = random.randint(3, 9)
                    filter_size = filter_size if filter_size % 2 == 1 else filter_size + 1  # 如果是偶數，將其增加1使其成為奇數
                    # 中值模糊處理
                    blurred_image = image.filter(ImageFilter.MedianFilter(size=filter_size))
                elif blur_type == "Mean":
                    # 均值模糊處理
                    blurred_image = image.filter(ImageFilter.BoxBlur(radius=random.randint(2, 10)))
                elif blur_type == "Bilateral":
                    # 將PIL圖片轉換為OpenCV格式
                    image_cv2 = np.array(image)
                    # 將圖片轉換為灰度圖像（雙邊濾波器需要灰度圖像）
                    gray_image = cv2.cvtColor(image_cv2, cv2.COLOR_RGB2GRAY)
                    # 應用雙邊濾波
                    blurred_image_cv2 = cv2.bilateralFilter(gray_image, d=9, sigmaColor=75, sigmaSpace=75)
                    # 將圖片轉換回PIL格式
                    blurred_image = Image.fromarray(blurred_image_cv2)
                elif blur_type == "Custom":
                    # 自定義模糊處理
                    blurred_image = image.filter(ImageFilter.Kernel((5, 5), custom_kernel.flatten()))
                
                # 構造輸出路徑
                output_path = os.path.join(target_folder, image_file)
                
                # 保存模糊後的圖片
                blurred_image.save(output_path)
                
                # 更新進度條
                pbar.update(1)


Processing CARRIER1_Door Damage - Custom: 100%|██████████████████████████████████████| 100/100 [00:01<00:00, 52.70it/s]
Processing CARRIER2_Panel dents or scratches - Gaussian: 100%|███████████████████████| 100/100 [00:01<00:00, 61.17it/s]
Processing CARRIER2_Panel dents or scratches - Median: 100%|█████████████████████████| 100/100 [00:41<00:00,  2.43it/s]
Processing CARRIER2_Panel dents or scratches - Mean: 100%|██████████████████████████| 100/100 [00:00<00:00, 111.19it/s]
Processing CARRIER2_Panel dents or scratches - Bilateral: 100%|██████████████████████| 100/100 [00:01<00:00, 93.97it/s]
Processing CARRIER2_Panel dents or scratches - Custom: 100%|█████████████████████████| 100/100 [00:01<00:00, 60.26it/s]
Processing CARRIER3_Panel aging, rust or breach - Gaussian: 100%|████████████████████| 100/100 [00:01<00:00, 65.50it/s]
Processing CARRIER3_Panel aging, rust or breach - Median: 100%|██████████████████████| 100/100 [00:38<00:00,  2.57it/s]
Processing CARRIER3_Panel aging, rust or